<a href="https://colab.research.google.com/github/vitorsr/ccd/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definição do Baseline

## Bibliotecas Utilizadas

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Extração de Dados

In [2]:
!wget -q -O ccd_2019.zip https://www.dropbox.com/s/7rriacb7c6vzf3m/ccd_2019.zip

!unzip ccd_2019.zip

Archive:  ccd_2019.zip
replace bdmep_meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!wget "https://onedrive.live.com/download?cid=BCBADB0836C97273&resid=BCBADB0836C97273%21115485&authkey=AJlb41BWz0YiBd8" -O 2019.zip
!unzip 2019.zip

--2019-10-26 01:25:44--  https://onedrive.live.com/download?cid=BCBADB0836C97273&resid=BCBADB0836C97273%21115485&authkey=AJlb41BWz0YiBd8
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://c9quda.bn.files.1drv.com/y4mkFThBmjwrK0wG66MVSsxiA8xUU_Y7JD49ysxJ2pl_3a8uEaBqLJZoQ_a34sZNW0hk2ZZ0GWOHpZBKpCEBe8wFqmkBaNlC-uL5VWidjZe4aknpqqmnyRTgR6aVNh5Xy1puhRP0KmdJcGoYBFG8sAm9EhbsLSDsExC6ufbPiqvWhPPzcggu__GZarDMfKfhGiNiu64hmF5pEG-XB2e8tJqHg/2019.zip?download&psid=1 [following]
--2019-10-26 01:25:45--  https://c9quda.bn.files.1drv.com/y4mkFThBmjwrK0wG66MVSsxiA8xUU_Y7JD49ysxJ2pl_3a8uEaBqLJZoQ_a34sZNW0hk2ZZ0GWOHpZBKpCEBe8wFqmkBaNlC-uL5VWidjZe4aknpqqmnyRTgR6aVNh5Xy1puhRP0KmdJcGoYBFG8sAm9EhbsLSDsExC6ufbPiqvWhPPzcggu__GZarDMfKfhGiNiu64hmF5pEG-XB2e8tJqHg/2019.zip?download&psid=1
Resolving c9quda.bn.files.1drv.com (c9quda.bn.files.1drv.com)... 1

In [0]:
directory = '2019'
files = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        files.append(filename)
        continue
    else:
        continue

In [0]:
df = pd.DataFrame()
for file in files:
    id_2019 = pd.read_csv(directory+'/'+file, delimiter=',')['id']
    id_2019.head()

## Leitura dos Dados

<br><b>variável | descrição | unidade </b></br>
<br>date | data e hora da coleta             -
<br> id | ID da estação de coleta             -
<br>prec | precipitação |       mm
<br>tair | temperatura do ar  | graus Celsius
<br>tw | temperatura de bulbo úmido |  graus Celsius
<br>tmax | temperatura máxima do ar  | graus Celsius
<br>tmin | temperatura mínima do ar  | graus Celsius
<br>urmax | umidade relativa máxima  |             %
<br>patm | pressão atmosférica         |    hPa
<br>pnmm | pressão atmosférica média ao nível do mar  |           hPa
<br>wd | direção do vento        |   graus
<br>wsmax  |  rajadas de vento    |         m/s
<br>n |  horas de sol          |     h
<br>cc |   cobertura de nuvens    |           -
<br>evap  |   evaporação        |      mm
<br>ur  |   umidade relativa          |     %
<br>ws   |   velocidade do vento       |      m/s

In [6]:
df = pd.read_csv("inmetr.csv")
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
0,1970-05-04 00:00:00,83010,NaN,NaN,NaN,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.75,NaN
1,1970-05-04 12:00:00,83010,NaN,25.3,24.0,NaN,23.7,90.0,1005.9,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2,1970-05-04 18:00:00,83010,NaN,29.2,27.1,NaN,NaN,85.0,1004.2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,1970-05-05 00:00:00,83010,NaN,25.0,27.0,32.6,NaN,92.0,1007.5,NaN,NaN,NaN,NaN,8.0,2.4,88.25,NaN
4,1970-05-05 12:00:00,83010,0.0,25.9,24.8,NaN,23.0,91.0,1006.4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [7]:
df_m = pd.read_csv("bdmep_meta.csv")
df_m.head()

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,83010,-68.733333,-11.016667,260.00,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,82704,-72.666667,-7.633333,170.00,Cruzeiro do Sul,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,82915,-67.800000,-9.966667,160.00,Rio Branco,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,82807,-70.766667,-8.166667,190.00,Tarauacá,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,83098,-36.166667,-10.150000,56.13,Coruripe,Alagoas,AL,America/Maceio,-3,America/Maceio,-3


Os dados apresentados abragem o período entre os anos de 1970 e 2018.

Filtrando apenas para o Mês de Agosto:

In [8]:
mask = (df['date'].str.split('-',n=2,expand = True)[1]=='08')
df = df.loc[mask]
df.size

20944306

Filtrando apenas as Estações ainda Existentes em 2019:

In [9]:
mask = (df['id'].isin(id_2019))
df = df.loc[mask]
df.size

70091

Extraindo o Ano da Coluna Date:

In [10]:
df['date'] = df['date'].str.split('-',n=1,expand = True)[1]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
5383623,08-01 00:00:00,83632,NaN,13.2,11.9,24.5,NaN,86.0,928.3,NaN,14.0,1.0,5.3,0.0,3.1,76.50,1.666667
5383624,08-01 12:00:00,83632,0.0,19.2,15.4,NaN,7.4,67.0,929.0,NaN,5.0,5.0,NaN,5.0,NaN,NaN,NaN
5383625,08-01 18:00:00,83632,NaN,23.4,17.1,NaN,NaN,53.0,926.0,NaN,0.0,0.0,NaN,2.0,NaN,NaN,NaN
5383626,08-02 00:00:00,83632,NaN,10.5,9.9,25.1,NaN,93.0,927.2,NaN,0.0,0.0,6.5,0.0,2.8,71.75,3.000000
5383627,08-02 12:00:00,83632,0.0,18.9,14.9,NaN,6.9,65.0,929.0,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN


Extraindo a Hora de Medição da Coluna Date:

In [11]:
df['date'] = df['date'].str.split(' ',n=1,expand = True)[0]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
5383623,08-01,83632,NaN,13.2,11.9,24.5,NaN,86.0,928.3,NaN,14.0,1.0,5.3,0.0,3.1,76.50,1.666667
5383624,08-01,83632,0.0,19.2,15.4,NaN,7.4,67.0,929.0,NaN,5.0,5.0,NaN,5.0,NaN,NaN,NaN
5383625,08-01,83632,NaN,23.4,17.1,NaN,NaN,53.0,926.0,NaN,0.0,0.0,NaN,2.0,NaN,NaN,NaN
5383626,08-02,83632,NaN,10.5,9.9,25.1,NaN,93.0,927.2,NaN,0.0,0.0,6.5,0.0,2.8,71.75,3.000000
5383627,08-02,83632,0.0,18.9,14.9,NaN,6.9,65.0,929.0,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN


Group By por Data e Estação:

In [12]:
df = df.groupby(by=['date','id']).mean()
df.head(1000)

,,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
date,id,,,,,,,,,,,,,,,
08-01,83632,0.140909,20.050847,15.944068,26.613889,9.495000,66.762712,927.569444,NaN,10.358974,1.316239,7.475000,2.338983,3.407500,70.147436,1.222222
08-02,83632,0.187234,20.673984,16.433333,27.202564,9.458140,66.349593,927.027778,NaN,10.382114,1.317073,7.459259,2.341463,3.497674,70.220238,1.408730
08-03,83632,0.002174,20.785714,16.455556,26.712821,11.016279,65.690476,926.491667,NaN,11.952381,1.440476,6.837037,2.888889,3.646512,69.113095,1.468254
08-04,83632,0.577778,20.804918,16.445082,27.229730,10.485366,65.622951,926.311111,NaN,13.909836,1.336066,7.364000,2.418033,3.600000,68.637500,1.275000
08-05,83632,0.262222,20.805833,16.513333,26.908108,10.619512,65.950000,926.652778,NaN,13.791667,1.391667,7.340000,2.400000,3.473171,69.062500,1.525000
08-06,83632,0.336957,20.898361,16.431967,27.281579,10.685714,64.491803,927.163889,NaN,13.918033,1.639344,7.361538,2.208333,3.559524,67.615854,1.552846
08-07,83632,0.344681,20.761600,16.401600,27.166667,9.988372,65.272000,927.211111,NaN,12.408000,1.520000,7.381481,1.992000,3.537209,69.904762,1.460318
08-08,83632,0.102128,21.050394,16.475397,27.597436,10.037209,64.464567,927.618919,1014.0,11.086614,1.343088,7.388889,2.165354,3.655814,67.375000,1.369896
08-09,83632,0.043478,21.012698,16.378571,27.369231,9.927907,63.714286,926.872222,NaN,11.730159,1.293651,7.110714,2.103175,3.760465,66.678571,1.325397


In [13]:
id_2019.head()

0    83632
1    83632
2    83632
3    83632
4    83632
Name: id, dtype: int64